## Configuring and Running Pre-Trade Compliance
In this example we demonstrate the configuration of several simple pre-trade rules and run them against a sample portfolio. Then we will raise some orders on that portfolio and re-check pre-trade compliance. Lastly, we will allocate partially against the newly created order and run a post-trade compliance. 

In [ ]:
# !pip3 install -U lusid-sdk finbourne-sdk-utils

## Imports

In [ ]:
import lusid
import os
from lusid.extensions import (
    SyncApiClientFactory,
    ArgsConfigurationLoader,
    EnvironmentVariablesConfigurationLoader,
    SecretsFileConfigurationLoader
)

import lusid.api as la
import lusidjam
import pandas as pd
from finbourne_sdk_utils import cocoon as cocoon
import lusid.models as lm
import datetime
from datetime import datetime, timedelta, time, date
import pytz
from pydantic.v1 import BaseModel, Field, StrictStr, validator
from lusid.models.upsert_compliance_rule_request import UpsertComplianceRuleRequest
from lusid.models.reference_list_request import ReferenceListRequest
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from collections import defaultdict

try:
    # Authenticate to SDK
    # Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
    secrets_path = os.getenv("FBN_SECRETS_PATH")
    # Run the Notebook locally using a secrets file (see https://support.lusid.com/docs/how-do-i-use-an-api-access-token-with-the-lusid-sdk)
    if secrets_path is None:
        secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")
        print()

    # Initiate an API Factory which is the client side object for interacting with LUSID APIs
    config_loaders=[
        ArgsConfigurationLoader(access_token = lusidjam.RefreshingToken(), app_name = "LusidJupyterNotebook"),
        EnvironmentVariablesConfigurationLoader()]
    api_factory = SyncApiClientFactory(config_loaders=config_loaders)
    
    print([api for api in dir(lusid.api) if "Api" in api])
except Exception as e:
    print(e)

In [ ]:
# define some APIs
properties_api = api_factory.build(la.PropertyDefinitionsApi)
referencelist_api = api_factory.build(la.ReferenceListsApi)
compliance_api = api_factory.build(la.ComplianceApi)
aggregation_api = api_factory.build(la.AggregationApi)
configuration_recipe_api = api_factory.build(la.ConfigurationRecipeApi)
portfolio_groups_api = api_factory.build(la.PortfolioGroupsApi)
portfolios_api = api_factory.build(la.PortfoliosApi)
instruments_api = api_factory.build(la.InstrumentsApi)
allocations_api = api_factory.build(lusid.api.AllocationsApi)
transaction_portfolios_api = api_factory.build(la.TransactionPortfoliosApi)

In [ ]:
# Specify a unique scope and code to segregate data in this tutorial from others
scope = "simplecompliance"
portfolio_code='EQUITY_UK'
portfolio_group_code='EQUITY_UK_GROUP'
portfolio_base_currency='USD'
print(f"'{scope}/{portfolio_code}' scope and code created.")

## 1. Create instruments, portfolio and transactions to work with


### 1.0 Load transaction data

In [ ]:
df = pd.read_csv("data/equity_transactions.csv")
df

### 1.1 Load instruments
loaded from df file

In [ ]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Isin": "ISIN",
        "Sedol": "sedol",
    },
    "required": {"name": "name"},
}

result = cocoon.cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
    property_columns=["sector"],
)
succ, failed, errors = cocoon.format_instruments_response(result)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

### 1.2 Create portfolio

In [ ]:
def create_portfolio(scope, portfolio_code, name):

    pf_df = pd.DataFrame(
        data=[
            {"portfolio_code": portfolio_code, "portfolio_name": name},
        ]
    )

    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "$USD",
        },
        "optional": {"created": "$2020-01-01T00:00:00+00:00"},
    }

    result = cocoon.cocoon.load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
    )
    #print(result['portfolios']['errors'][0])

    succ, failed = cocoon.format_portfolios_response(result)
    display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}]))

In [ ]:
create_portfolio(scope, portfolio_code, "LUSID's top 10 FTSE stock portfolio")

### 1.3 Create portfolio group

In [ ]:
def create_portfolio_group(portfolio_groups_api, scope, code, portfolios):

    portfolio_creation_date =datetime(2020, 1, 1, tzinfo=pytz.utc)

    try:
        portfolio_groups_api.delete_portfolio_group(scope=scope, code=code)
    except:
        pass

    group_request = lm.CreatePortfolioGroupRequest(
        code=code,
        display_name=code,
        values=portfolios,
        sub_groups=None,
        description=None,
        created=portfolio_creation_date,
    )

    portfolio_group = portfolio_groups_api.create_portfolio_group(
        scope=scope, create_portfolio_group_request=group_request
    )

    return portfolio_group

In [ ]:
result = create_portfolio_group(portfolio_groups_api, scope, portfolio_group_code, [lm.ResourceId(scope=scope, code=portfolio_code)])

### 1.4 Load transactions

In [ ]:
transaction_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
    },
    "required": {
        "code": f"${portfolio_code}",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": f"${portfolio_base_currency}",
        "settlement_date": "txn_settle_date",
    },
    "optional": {},
    "properties": [],
}

result = cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
    properties_scope=scope,
)
succ, failed = cocoon.format_transactions_response(result)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

### 1.5 Load quotes

In [ ]:
quotes_df = pd.read_csv("data/equity_quotes.csv")
# Compliance runs at latest asat
quotes_df["quote_date"] = quotes_df["quote_date"].apply(
    lambda s: datetime.now(pytz.UTC).strftime("%d-%b-%y")
)
quotes_df

In [ ]:
quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$Isin",
    "quote_id.effective_at": "quote_date",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "ISIN",
    "metric_value.unit": "$USD",
}


quotes_mapping["quote_id.quote_series_id.field"] = "$mid"
quotes_mapping["metric_value.value"] = "mid"
result = cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=quotes_df,
    mapping_required=quotes_mapping,
    mapping_optional={},
    file_type="quotes",
)
succ, failed, errors = cocoon.format_quotes_response(result)
display(
    pd.DataFrame(
        data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
    )
)

### 1.6 Create recipes

In [ ]:
# Create a recipe to perform a valuation
configuration_recipe = lm.ConfigurationRecipe(
    scope="User",
    code="valuation_recipe_mid",
    market=lm.MarketContext(
        market_rules=[
            # define how to resolve the quotes
            lm.MarketDataKeyRule(
                key="Quote.Isin.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
            ),
        ],
        options=lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="Isin",
            default_scope=scope,
        ),
    ),
    pricing=lm.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": True},
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=lm.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

### 1.7 Run a test valuation

In [ ]:
price_field = "mid"

quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$Isin",
    "quote_id.effective_at": "quote_date",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "ISIN",
    "metric_value.unit": "$USD",
}


quotes_mapping["quote_id.quote_series_id.field"] = f"${price_field}"
quotes_mapping["metric_value.value"] = price_field

result = cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=quotes_df,
    mapping_required=quotes_mapping,
    mapping_optional={},
    file_type="quotes",
)

succ, failed, errors = cocoon.format_quotes_response(result)
display(
    pd.DataFrame(
        data=[
            {
                f"[{price_field}] success": len(succ),
                "failed": len(failed),
                "errors": len(errors),
            }
        ]
    )
)

In [ ]:
def generate_valuation_request(valuation_effectiveAt, price_field):

    # Create the valuation request
    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(
            scope="User", code="valuation_recipe" + "_" + price_field
        ),
        metrics=[
            lm.AggregateSpec(key = "Instrument/default/Name", op = "Value"),
            lm.AggregateSpec(key = "Valuation/PvInReportCcy", op = "Proportion"),
            lm.AggregateSpec(key = "Valuation/PvInReportCcy", op = "Sum"),
            lm.AggregateSpec(key = "Holding/default/Units", op = "Sum"),
            lm.AggregateSpec(key = "Aggregation/Errors", op = "Value"),
        ],
        group_by=["Instrument/default/Name"],
        #filters=[lm.PropertyFilter('Instrument/default/Name', 'NotEquals', 'USD'),
        #        lm.PropertyFilter('Instrument/default/Name', 'NotEquals', 'USD Cash')
        #],
        portfolio_entity_ids=[
            lm.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=lm.ValuationSchedule(
            effective_at=valuation_effectiveAt.isoformat()
        ),
    )

    return valuation_request

In [ ]:
# we will use the mid price
aggregation = aggregation_api.get_valuation(
    valuation_request=generate_valuation_request(datetime.now(pytz.UTC), "mid")
)
pd.DataFrame(aggregation.data)

## 2. Define properties and supporting data
Adding properties to portfolios will allow lookup portfolios by property value, using the propertyKey argument to decorate the portfolios with the desired property/properties, and then filter to specify which portfolios should be returned.

In this example, since there is only one fund within this portgroup, we will not use properties to do any portfolio filtering. Therefore, we will create a property called 'placeholder' and not assign any value to the portfolio with this property.

We will then create an instrument property that classifies the holding as ftse100 or not and use that in our compliance rules.

In [ ]:
def create_property_definition(properties_api, domain, scope, code, data_type):
    properties_api.create_property_definition(
        create_property_definition_request=lm.CreatePropertyDefinitionRequest(
            domain=domain,
            scope=scope,
            code=code,
            display_name=code,
            life_time="Perpetual",
            value_required=False,
            data_type_id=lm.resource_id.ResourceId(scope="system", code=data_type),
        )
    )

In [ ]:
try:
    create_property_definition(
        properties_api, "Portfolio", scope, "Placeholder", "string"
    )
except:
    pass

In [ ]:
try:
    create_property_definition(properties_api, "Instrument", scope, 'ftse100', "string")
except:
    pass

### 2.1 Decorate portfolio and instrument with properties


In [ ]:
response = portfolios_api.upsert_portfolio_properties(
    scope=scope,
    code=portfolio_code,
    request_body={
        "Portfolio/{}/Placeholder".format(scope): lm.ModelProperty(
            key="Portfolio/{}/Placeholder".format(scope),
            value=lm.PropertyValue(label_value="PlaceholderValue"),
        )
    },
)
# Placeholder will be used later on to setting up compliance rules. We are creating this property

In [ ]:
#we are doing a simple filtering such that if the holding in our portfolio is BHP, its not ftse100. 
for i in df.index:
    if df.loc[i, 'name'] =='BHP':
        df.loc[i, 'ftse100'] = 'No'
    else:
        df.loc[i, 'ftse100'] = 'Yes'

In [ ]:
requests = []

for row in df.iterrows():
    # Collect our instrument data from the row, note row[0] is the index of the row e.g. 3
    instrument = row[1]
    
    # Create our ftse100 classification property for this instrument
    asset_class_property1 = lm.ModelProperty(
        key=f"Instrument/{scope}/ftse100",
        value=lm.PropertyValue(
            label_value=instrument['ftse100'])
    )    

    # Build our request to set our property
    requests.append(
        lm.UpsertInstrumentPropertyRequest(
            identifier_type='ClientInternal',
            identifier=instrument['instrument_id'],
            properties=[asset_class_property1])
    )

# Call the LUSID API to add our property across all instruments
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments_properties(scope='default',
    upsert_instrument_property_request=requests)

### 2.2 Create reference lists

We need a couple of empty lists to allow pass-through behaviour. Note that empty list passed to a filter step in compliance rule setup means that we disable this filter step. please refer to 3.1 for the logic behind this. 
https://support.lusid.com/knowledgebase/article/KA-02248/en-us

A reference list in LUSID is a repository of data that can contain:

- A series of strings; for example if you want to store a list of asset classes.
- A series of portfolios or portfolio groups; for example if you want to store a list of portfolios to exclude from a compliance rule.
- A series of instruments; for example if you want to store a list of instruments to exclude from a particular compliance rule.
- A series of address keys; for example if you want to store a list of instrument properties that should exist to pass a compliance check.
- A series of decimals; for example if you wanted to store a list of upper and lower bounds that an order should remain within.

All reference lists in LUSID have a scope and a code forming a unique ID. Currently, you can pass one or more reference list IDs into various LUSID compliance rules to ensure the rule checks against the values in the list. For example, you can:

- Set up a compliance rule that checks orders do not comprise of instruments of a particular set of asset classes.
- Set up a compliance rule that checks orders contain a list of required properties, such as the country of origin.
- Specify a reference list of portfolios within a compliance rule to exclude those in the list from the rule.


In [ ]:
request = ReferenceListRequest(
    id=lm.ResourceId(scope=scope, code="empty-list"),
    name="empty string list",
    description="All asset classes",
    tags=[],
    reference_list=lm.StringList(reference_list_type="StringList", values=[]),
)

empty_list_response = referencelist_api.upsert_reference_list_with_http_info(
    reference_list_request=request
)

request = ReferenceListRequest(
    id=lm.ResourceId(scope=scope, code="empty-portfolio-list"),
    name="empty portfolioid list",
    description="all portfolios",
    tags=[],
    reference_list=lm.PortfolioIdList(reference_list_type="PortfolioIdList", values=[]),
)

empty_portfolioid_list_response = (
    referencelist_api.upsert_reference_list_with_http_info(
        reference_list_request=request
    )
)

# we will add mining here and will exclude mining from the portfolio holding later on

request = ReferenceListRequest(
    id=lm.ResourceId(scope=scope, code="restricted-sectors"),
    name="restricted sectors list",
    description="No mining firms",
    tags=[],
    reference_list=lm.StringList(reference_list_type="StringList", values=["Mining"]),
)

empty_portfolioid_list_response = (
    referencelist_api.upsert_reference_list_with_http_info(
        reference_list_request=request
    )
)

#we will also add a yes as accepted value for our binary classification ftse100.
request = ReferenceListRequest(
    id=lm.ResourceId(scope=scope, code="ft100-classified"),
    name="ftse100 binary",
    description="only ftse100 firms",
    tags=[],
    reference_list=lm.StringList(reference_list_type="StringList", values=["Yes"]),
)

empty_portfolioid_list_response = (
    referencelist_api.upsert_reference_list_with_http_info(
        reference_list_request=request
    )
)

## 3. Setup compliance rules

### 3.1 A simple PV contribution threshold check
Here we are creating a 'PercentCheck' template rule that is using standard variation. 

In standard variation, you will see 2 filtered properties that are mandatatory to populate. As mentioned previously, if the property is not populated with any value, such as placeholder property and the reference list is empty, then we effectively disable the filter setup.

In our example, for instance, our first filtering is that this rule will only apply to firms that are within ftse100 and our second filtering is an empty one. If there is no need to use any filtering, then we could create the rule as simple variation since in this format, you do not specify property filtering and portfolio exclusion

Note that the compliance rulkes can be configured pre-trade or post-trade. with pre-trade screening incoming orders for the given scope and protgroup. please refer to this page for a general overview of Lusid OMS. https://support.lusid.com/knowledgebase/article/KA-02107/en-us


In [ ]:
pvInReportCcy = lm.AddressKeyComplianceParameter(
    value="Valuation/PvInReportCcy", description="passed_validation", compliance_parameter_type="AddressKeyComplianceParameter"
)
#pvInReportCcy._value = "Valuation/PvInReportCcy"

upsert_compliance_rule_request = UpsertComplianceRuleRequest(
    id=lm.ResourceId(scope=scope, code="rule-1"),
    name="threshold-pv-check",
    description="Simple threshold PV check",
    template_id=lm.ResourceId(scope="system", code="PercentCheck"),
    variation="standard",
    portfolio_group_id=lm.ResourceId(scope=scope, code=portfolio_group_code),
    active=True,
    parameters={
        "Metric": pvInReportCcy,
        "UpperBound": lm.DecimalComplianceParameter(
            value=10, compliance_parameter_type="DecimalComplianceParameter"
        ),
        "LowerBound": lm.DecimalComplianceParameter(
            value=-1.0, compliance_parameter_type="DecimalComplianceParameter"
        ),
        "UpperWarning": lm.DecimalComplianceParameter(
            value=30.0, compliance_parameter_type="DecimalComplianceParameter"
        ),
        "LowerWarning": lm.DecimalComplianceParameter(
            value=-1.0, compliance_parameter_type="DecimalComplianceParameter"
        ),
        "FirstFilterPropertyKey": lm.PropertyKeyComplianceParameter(
            value=f"Instrument/{scope}/ftse100",
            compliance_parameter_type="PropertyKeyComplianceParameter",
        ),
        "FirstFilterPermittedValuesList": lm.StringListComplianceParameter(
            value=lm.ResourceId(scope="simplecompliance", code="ft100-classified"),
            compliance_parameter_type="StringListComplianceParameter",
        ),
        "SecondFilterPropertyKey": lm.PropertyKeyComplianceParameter(
            value=f"Portfolio/{scope}/Placeholder",
            compliance_parameter_type="PropertyKeyComplianceParameter",
        ),
        "SecondFilterPermittedValuesList": lm.StringListComplianceParameter(
            value=lm.ResourceId(scope="simplecompliance", code="empty-list"),
            compliance_parameter_type="StringListComplianceParameter",
        ),
        "GroupingPropertyKey": lm.PropertyKeyComplianceParameter(
            value=f"Instrument/default/Name",
            compliance_parameter_type="PropertyKeyComplianceParameter",
        ),
        "Excludes": lm.PortfolioIdListComplianceParameter(
            value=lm.ResourceId(scope="simplecompliance", code="empty-portfolio-list"),
            compliance_parameter_type="PortfolioIdListComplianceParameter",
        ),
    },
    properties={},
)

compliance_api.upsert_compliance_rule(
    upsert_compliance_rule_request=upsert_compliance_rule_request
)

### 3.2 A restricted sector check

We created 'restricted-sectors' with Mining as the value. In this rule, we are creating a 'PropertyValueNotInList' template where we look at the sector of the instruments and if the instrument is in 'Mining', then this rule will flag. 

- standard-not-in-list: No instruments can have the value of 'PropertyKey' in 'ExclusivePropertyList'
- standard-in-list: Instruments must have the value of 'PropertyKey' in 'ExclusivePropertyList'

In [ ]:
upsert_compliance_rule_request = UpsertComplianceRuleRequest(
    id=lm.ResourceId(scope=scope, code="rule-2"),
    name="restricted-sector-check",
    description="Simple sector restriction",
    template_id=lm.ResourceId(scope="system", code="PropertyValueNotInList"),
    variation="standard-not-in-list",
    portfolio_group_id=lm.ResourceId(scope=scope, code=portfolio_group_code),
    active=True,
    parameters={
        "PropertyKey": lm.PropertyKeyComplianceParameter(
            value=f"Instrument/simplecompliance/sector",
            compliance_parameter_type="PropertyKeyComplianceParameter",
        ),
        "ExclusivePropertyList": lm.StringListComplianceParameter(
            value=lm.ResourceId(scope=scope, code="restricted-sectors"),
            compliance_parameter_type="StringListComplianceParameter",
        ),
    },
    properties={},
)

compliance_api.upsert_compliance_rule(
    upsert_compliance_rule_request=upsert_compliance_rule_request
)

## 4. Run Compliance Rules

Currently the API enables us to run compliance as at now, rather than for a specific date.

In [ ]:
run_response = compliance_api.run_compliance(run_scope=scope,rule_scope=scope,is_pre_trade=True,recipe_id_scope='User',recipe_id_code='valuation_recipe_mid')
run_code = run_response.run_id.code

print(f"Compliance run {run_response.run_id.scope}/{run_response.run_id.code} completed.")
print(f"Instigated: {run_response.instigated_at}, completed: {run_response.completed_at}")

## 5. Analyse the compliance result¶
### 5.1 Identify rule-level failures
At a very coarse level we can determine which rules have been breached in this compliance run. We can also see a list of orders and portfolios affected by the breach.

The list of orders might well be wider than initially expected. This is because LUSID compliance makes no assumptions about which orders take precedence; it takes a holistic, contingent view about compliance taking into account all orders and existing positions.

Here you can see that rule-1 has failed, and one portfolio is affected by the failure. We'll drilldown into this rule result in the following section.

In [ ]:
def rule_level_dataframe(run_summary):
    # Use the first result as a way of generating overall headers
    h = ["", "", "", "", ""]
    c = ["Rule", "Rule Description", "Status", "Affected Orders", "Affected Portfolios"]

    df = pd.DataFrame([c], columns=h)

    new_labels = pd.MultiIndex.from_arrays([df.columns, df.iloc[0]], names=["", ""])
    df = df.set_axis(new_labels, axis=1).iloc[1:]

    # Now build a row per result
    for d in run_summary.details:
        r = [
            f"{d.rule_id.scope}/{d.rule_id.code}",
            d.rule_description,
            d.status,
            len(d.affected_orders),
            len(d.affected_portfolios_details),
        ]

        df.loc[len(df)] = r

    return df

In [ ]:
run_summary = compliance_api.get_decorated_compliance_run_summary(
    scope=scope, code=run_code
)

df = rule_level_dataframe(run_summary)

print(
    f"Rule-level results for run {run_summary.run_id.scope}/{run_summary.run_id.code}."
)
display(df)

### 5.2 Drilldown into specific rule results

#### 5.2.1 Simple PV threshold breach

`rule-1` failed the compliance check overall. This rule is a simple 10% PV contribution threshold check often used to control portfolio diversification, so a failure indicates that one or more instruments contributed more than 10%. Taking a look at the drilldown data for this rule we can see that `Rightmove`, `vodafone` and `Anglo American plc` in the `EQUITY_UK` portfolio each contribute more than 10%.

The rule is structured to
- start with all holdings in portfolio group 
- filter out contributions *for portfolios in an excluded portfolios list*
- group contributions *by portfolio id*
- two filter steps, here configured to include only ftse100 holdings
- further group contributions for each portfolio *by Name*
- finally, compare a pair of Results Used (Valuation/PVInReportCcy compared to portfolio-level Valuation/PVInReportCcy) *(checking that one is less than 10% of the other)*

This information is represented by the Lineage for each drilldown row; the Lineage can be used to get a high-level view of what's caused a rule breach, and a pointer to where to start more detailed investigations if needed.

In [ ]:
rule_result1 = compliance_api.get_compliance_rule_result(
    run_scope=scope, run_code=run_code, rule_scope=scope, rule_code="rule-1"
)


filtered_breakdowns1 = [
    breakdown
    for breakdown in rule_result1.rule_result.rule_breakdown
    if getattr(breakdown, "group_status", "") == "Failed"
]
rule_result1_df = lusid_response_to_data_frame(filtered_breakdowns1)

display(rule_result1_df[[
            "groupStatus",
            "resultsUsed.Valuation/PvInReportCcy",
            "resultsUsed.Portfolios.Valuation/PvInReportCcy",
            "lineage.1.label",
            "lineage.2.subLabel",
            "lineage.5.subLabel",
            "lineage.5.information",
    ]])

### 5.2.2 Restricted sector breach
Rule-2 failed the compliance check overall. This rule prevents trade in a restricted set of sectors (specifically just Mining in this case), so a failure indicates that one or more holdings across the whole portfolio group are of an instrument from a restricted sector.

In [ ]:
rule_result2 = compliance_api.get_compliance_rule_result(
    run_scope=scope, run_code=run_code, rule_scope=scope, rule_code="rule-2"
)

filtered_breakdowns2 = [
    breakdown
    for breakdown in rule_result2.rule_result.rule_breakdown
    if getattr(breakdown, "group_status", "") == "Failed"
]

rule_result2_df = lusid_response_to_data_frame(filtered_breakdowns2)
display(rule_result2_df[["groupStatus", "lineage.1.subLabel"]])

## 6 Upsert an order to show its effect
We will now upsert an order and run compliance rule-1 against that. It should flag as i have an order that breaches the PV threshold.

In [ ]:
orders_df = pd.read_csv("data/live_orders.csv")
orders_df

In [ ]:
# generating the order via order API
order_requests = defaultdict(list)
order_sets = defaultdict(list)
responses = []

for index, order in orders_df.iterrows():

    request = lm.OrderRequest(
        id=lm.ResourceId(scope=scope, code=order["order_id"]),
        quantity=order["quantity"],
        side=order["side"],
        instrument_identifiers={
            "Instrument/default/ClientInternal": order["client_internal"]
        },
        properties={},
        portfolio_id=lm.ResourceId(scope=scope, code=portfolio_code),
        state=order["state"],
        type=order["type"],
        price=lm.CurrencyAndAmount(
            amount=0 if pd.isna(order["price"]) else order["price"],
            currency=order["currency"],
        ),
        limit_price=(
            lm.CurrencyAndAmount(
                amount=order["limit_price"], currency=order["limit_currency"]
            )
            if not pd.isna(order["limit_price"])
            and not pd.isna(order["limit_currency"])
            else None
        ),
        stop_price=(
            lm.CurrencyAndAmount(
                amount=order["stop_price"], currency=order["stop_currency"]
            )
            if not pd.isna(order["stop_price"]) and not pd.isna(order["stop_currency"])
            else None
        ),
    )

    request = lm.OrderSetRequest(order_requests=[request])

    response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
        order_set_request=request
    )

    responses.append(response.values[0])

attributes = [
    (
        o.id.code,
        o.instrument_identifiers["Instrument/default/ClientInternal"],
        o.lusid_instrument_id,
        o.side,
        o.type,
        o.state,
        o.quantity,
        o.price.amount,
        o.price.currency,
        o.limit_price.amount if o.limit_price is not None else "N/A",
        o.limit_price.currency if o.limit_price is not None else "N/A",
    )
    for o in responses
]

pd.DataFrame(
    attributes,
    columns=[
        "order_id",
        "client_internal",
        "lusid_instrument_id",
        "side",
        "type",
        "state",
        "quantity",
        "price",
        "currency",
        "lim px",
        "lim ccy",
    ],
).style.format({"quantity": "{:20,.0f}", "price": "{:20,.2f}"})

### 6.2 Simple PV threshold breach
The orders that we've upserted will fail rule-1 as the order PV will exceed fund's 10% PV rule. 
We can see that with the addition of this new orders, number of instruments that are failing the PV valuation threshold increased from 3 to 4.

In [ ]:
order_result = compliance_api.run_compliance(
    is_pre_trade=True,
    recipe_id_scope="User",
    recipe_id_code="valuation_recipe_mid",
    run_scope=scope,
    rule_scope=scope,
)
run_code_order = order_result.run_id.code

In [ ]:
order_result = compliance_api.get_compliance_rule_result(
    run_scope=scope, run_code=run_code_order, rule_scope=scope, rule_code="rule-1"
)
filtered_breakdowns_orders = [
    breakdown
    for breakdown in order_result.rule_result.rule_breakdown
    if getattr(breakdown, "group_status", "") == "Failed"
]


order_result_df = lusid_response_to_data_frame(filtered_breakdowns_orders)
display(
    order_result_df[
        [
            "groupStatus",
            "resultsUsed.Valuation/PvInReportCcy",
            "resultsUsed.Portfolios.Valuation/PvInReportCcy",
            "lineage.1.label",
            "lineage.2.subLabel",
            "lineage.5.subLabel",
            "lineage.5.information",
        ]
    ]
)

## 7 Upsert a trade to show its effect
we will now upsert an allocation and run compliance against that. I will first post a small allocation against ORD005 which is a buy of 1 million shares

In [ ]:
allocations_df = pd.read_csv("data/iag_allocations.csv")
allocations_df

In [ ]:
allocation_requests = defaultdict(list)
allocation_sets = defaultdict(list)
responses = []

for index, allocation in allocations_df.iterrows():

    portfolio = portfolio_code

    request = lm.AllocationRequest(
        id=lm.ResourceId(scope=scope, code=allocation["allocation_id"]),
        allocated_order_id=lm.ResourceId(
            scope=scope, code=allocation["originating_order"]
        ),
        quantity=allocation["quantity"],
        side=allocation["side"],
        instrument_identifiers={
            "Instrument/default/ClientInternal": allocation["client_internal"]
        },
        properties={},
        portfolio_id=lm.ResourceId(scope=scope, code=portfolio_code),
        state=allocation["state"],
        type=allocation["type"],
        price=lm.CurrencyAndAmount(
            amount=allocation["price"], currency=allocation["currency"]
        ),
    )

    request = lm.AllocationSetRequest(allocation_requests=[request])

    response = allocations_api.upsert_allocations(allocation_set_request=request)

    responses.append(response.values[0])

attributes = [
    (
        o.id.code,
        o.instrument_identifiers["Instrument/default/ClientInternal"],
        o.lusid_instrument_id,
        o.side,
        o.type,
        o.state,
        o.quantity,
        o.price.amount,
    )
    for o in responses
]

pd.DataFrame(
    attributes,
    columns=[
        "allocation_id",
        "client_internal",
        "lusid_instrument_id",
        "side",
        "type",
        "state",
        "quantity",
        "price",
    ],
).style.format({"quantity": "{:20,.0f}", "price": "${:20,.2f}"})

### 7.2 Simple PV threshold pass 
As this allocation is of quantity 500, it should not breach the 10% PV rule and we expect this to pass rule-1.

In [ ]:
alloc_result = compliance_api.run_compliance(
    is_pre_trade=False,
    recipe_id_scope="User",
    recipe_id_code="valuation_recipe_mid",
    run_scope=scope,
    rule_scope=scope,
)
run_code_alloc = alloc_result.run_id.code

In [ ]:
alloc_result = compliance_api.get_compliance_rule_result(
    run_scope=scope, run_code=run_code_alloc, rule_scope=scope, rule_code="rule-1"
)
filtered_breakdowns_alloc = [
    breakdown
    for breakdown in alloc_result.rule_result.rule_breakdown
    if getattr(breakdown, "group_status", "") == "Passed"
]

# Print the filtered breakdowns
alloc_result_df = lusid_response_to_data_frame(filtered_breakdowns_alloc)
display(
    alloc_result_df[
        [
            "groupStatus",
            "resultsUsed.Valuation/PvInReportCcy",
            "resultsUsed.Portfolios.Valuation/PvInReportCcy",
            "lineage.1.label",
            "lineage.2.subLabel",
            "lineage.5.subLabel",
            "lineage.5.information",
        ]
    ]
)